# Decisoin Tree


* 의사결정에 영향을 미치는 속성을 값에 따라 분기하여, 최종 결정에 이르도록 함 (사다리타기와 유사).

## ID3 algorithm

* input: Training Data, Attribute List
  * 학습데이터
  * 속성목록
* output: decision tree

Generate_decision_tree(Training Data, Attribute List)
* create a node N 시작 노드
* if samples are all of the same class C then
    return N as a leaf node labeled with class C; 모두 같은 클래스인 경우 분기하여 노드
* if attribute-list is empty then
    return N as a leaft node labeled with majority voting; 분기할 속성이 남아 있지 않으면 최빈으로 노드
* from among attributes in the attribute-list,
    select test attribute that leads to the highest information gain
    label node N with the test-attribute; IG가 가장 높은 속성을 선택하여 노드
* for each known value ai of test-attribute //partition the samples
    * grow a branch from node N for the condition test-attribute = ai 분기
    * let si be the set of samples in the samples for which test-attribute = ai 데이터분할
    * if si is empty then
        attach a leaf labelled with majority voting; 분할한 데이터가 공집합이면 최빈
    * else
        attach the node returned by Generate_decision_tree(si, Attribute List - test attribute); 재귀적으로 함수를 실행하고 그 결과 값을 받아서 노드
}

## 프로그래밍

1. 데이터준비
2. 엔트로피 계산
3. Information Gain 계산
4. decision tree 구조 만듦
5. 분류

## Shannon entorpy

* 정보이론 (Shannon, 1948)
* Entropy H(S)는 데이터에 포함된 불확실성, 엔트로피가 클수록 불확실성이 높음.
* log2를 취하면서 그 승수가 정보를 표현하는 bit수를 의미.
즉 4의 log2 값은 2 ($2^2$), 4개의 정보 값을 표현하려면 2 bits 필요.

참고
* scipy.stats.entropy
* Gini impurity

$$
\begin{align}
H(S) &= - \sum_{i=1}^{n}\ p_i\ log_2\ p_i\\
     &= - p_1 log_2\ p_1 - p_2 log_2\ p_2\ \ldots
\end{align}
$$

위 식에서 $p_i$는 사건i가 발생할 확률을 의미.
동전을 던지는 경우, 모두 앞면 또는 모두 뒷면은 entropy가 0 (즉 불확실성이 없는 purity)
반대로 반반씩 섞여 있는 경우 entropy는 가장 크다.
즉 엔트로피가 클수록 정보가 혼재되어 있다는 의미.
가장 유용한 정보는 엔트로피를 가장 많이 감소시키는 것.

* H( (0.5,0.5) ) = -0.5 x log_2 0.5 - 0.5 x log_2 0.5 = 1
* H( (0.5,0.5) ) = -0.7 x log_2 0.7 - 0.3 x log_2 0.3 = 0.88
* H( (0.5,0.5) ) = -1.0 x log_2 1.0 - 0.0 x log_2 0.0 = 0

In [6]:
import math
print [-p*math.log(p,2) for p in [0.5,0.5]]
print [-p*math.log(p,2) for p in [0.7,0.3]]
try:
    [-p*math.log(p,2) for p in [1.0,0.0]]
except:
    print "math domain error"

[0.5, 0.5]
[0.3602012209808308, 0.5210896782498619]
math domain error


### 문자열의 예

* 엔트로피는 부호화에 필요한 문자 당 최소 평균 이진값

In [1]:
# compute entropy with a string example
import numpy as np
import math
str = 'aabcddddefffg'
# 1) count frequency
allChars=list(str)
#uniqueChars=set(allChars)
#dictionary to save frequencies of all characters
tokenVector=dict()
for token in allChars:
    # increase if the key exists
    if tokenVector.has_key(token):
        tokenVector[token]+=1
    else:
        tokenVector[token]=1
# 2) computes entropy
entropy=0
allFreq=float(len(allChars))
for key in tokenVector.iterkeys():
    freq=tokenVector[key]
    prob=float(freq)/allFreq
    ent=-prob * math.log(prob,2)
    entropy=entropy+ent
    print "{0} 빈도 {1} 확률 {2} 엔트로피 {3} 엔트로피누적 {4}".format(key,freq,prob,ent,entropy)

# 묶어서 함수로
# yes가 2/5 no가 3/5인 경우 (교재 p.42)
S=np.array( (0.4, 0.6))
entropy=[ -p*math.log(p,2) for p in S ]
def getEntropy(data):
    entropy=[-p*math.log(p,2) for p in data]
    return sum(entropy)
#0.97095059
print "entropy={0}\nfunct={1}".format(entropy,getEntropy(S))

a 빈도 2 확률 0.153846153846 엔트로피 0.415452264329 엔트로피누적 0.415452264329
c 빈도 1 확률 0.0769230769231 엔트로피 0.284649209088 엔트로피누적 0.700101473417
b 빈도 1 확률 0.0769230769231 엔트로피 0.284649209088 엔트로피누적 0.984750682505
e 빈도 1 확률 0.0769230769231 엔트로피 0.284649209088 엔트로피누적 1.26939989159
d 빈도 4 확률 0.307692307692 엔트로피 0.523212220966 엔트로피누적 1.79261211256
g 빈도 1 확률 0.0769230769231 엔트로피 0.284649209088 엔트로피누적 2.07726132165
f 빈도 3 확률 0.230769230769 엔트로피 0.488187050174 엔트로피누적 2.56544837182
entropy=[0.52877123795494485, 0.44217935649972373]
funct=0.970950594455


## information gain

* 불확실성이 가장 낮은 속성으로 분기하기 위한 방법. 가장 높은 IG를 선택하여 분기함.
* 속성선정값 (attribute selection measure) 또는 분기적합도 (measure of the goodness of split)

$$
IG(A,S) = H(S) - \sum_{i=1}^m\ p(i)\ H(i)
$$

* 각 속성의 값으로, 그 발생확률을 계산 (위 식 $p(i)$)
* 그 속성 값의 엔트로피를 계산 (H(i)
* 위 1과 2를 가중하여 해당 속성의 엔트로피를 계산 (위 식의 우측 항)
* 기본엔트로피 (H(s))에서 공제하여 Information Gain 

## 예제

* 속성 4개 열, 속성 값은 문자열
* 클래스 마지막 열
* 테이블로 나타내면:

| age | has_job | own_house | credit_rating | class |
|-----|---------|-----------|---------------|-------|
| young | false | false | fair | no |

* numpy로 계산 (List로 처리하는 것과 비교)

## 1. 데이터 준비

In [2]:
S=np.array([['young', 'false', 'false', 'fair', 'No'],
       ['young', 'false', 'false', 'good', 'No'],
       ['young', 'true', 'false', 'good', 'Yes'],
       ['young', 'true', 'true', 'fair', 'Yes'],
       ['young', 'false', 'false', 'fair', 'No'],
       ['middle', 'false', 'false', 'fair', 'No'],
       ['middle', 'false', 'false', 'good', 'No'],
       ['middle', 'true', 'true', 'good', 'Yes'],
       ['middle', 'false', 'true', 'excellent', 'Yes'],
       ['middle', 'false', 'true', 'excellent', 'Yes'],
       ['old', 'false', 'true', 'excellent', 'Yes'],
       ['old', 'false', 'true', 'good', 'Yes'],
       ['old', 'true', 'false', 'good', 'Yes'],
       ['old', 'true', 'false', 'excellent', 'Yes'],
       ['old', 'false', 'false', 'fair', 'No']], 
      dtype='|S9')
print S

[['young' 'false' 'false' 'fair' 'No']
 ['young' 'false' 'false' 'good' 'No']
 ['young' 'true' 'false' 'good' 'Yes']
 ['young' 'true' 'true' 'fair' 'Yes']
 ['young' 'false' 'false' 'fair' 'No']
 ['middle' 'false' 'false' 'fair' 'No']
 ['middle' 'false' 'false' 'good' 'No']
 ['middle' 'true' 'true' 'good' 'Yes']
 ['middle' 'false' 'true' 'excellent' 'Yes']
 ['middle' 'false' 'true' 'excellent' 'Yes']
 ['old' 'false' 'true' 'excellent' 'Yes']
 ['old' 'false' 'true' 'good' 'Yes']
 ['old' 'true' 'false' 'good' 'Yes']
 ['old' 'true' 'false' 'excellent' 'Yes']
 ['old' 'false' 'false' 'fair' 'No']]


## 2. 엔트로피 계산

In [12]:
#학습데이터 첫째 속성(나이)의 엔트로피계산을 위한 확률 구함.
#키를 찾아, 빈도를 계산
a=S[:,0]
keys=np.unique(a)
bins=keys.searchsorted(S[:,0])
freq=np.bincount(bins)
print "나이{0} 빈도{1}".format(keys,freq)

나이['middle' 'old' 'young'] 빈도[5 5 5]


In [5]:
#함수로 구현
#in: array (n x 1) (속성 컬럼)
#out: array(2 x n) (키의 빈도 row1=keys, row2:frequencies)
def findKeyCounts(data):
    #find keys in nominal data
    keys=np.unique(data)
    #find indices for data (if sorted)
    bins=keys.searchsorted(data)
    #count for each key index
    #count returned as strings. conversion does not work?
    #All items in a numpy array have to have the same dtype.
    #possibly use a numpy recarray
    #return np.vstack([keys,np.bincount(bins).astype(np.int)])
    return np.vstack([keys,np.bincount(bins)])
print "첫째 속성의 키/빈도수 {0}".format(findKeyCounts(S[:,0]))
#pandas를 사용해서 구하려면, pd.value_counts(a[:,0])

#전체 데이터에 대해 엔트로피를 계산하면
#1) 확률 계산
#Yes: 9/15 No: 6/15
print "마지막 컬럼 클래스의 키/빈도수 {0}".format(findKeyCounts(S[:,-1]))

#확률을 구하려면, 전체빈도수를 계산.
kc=findKeyCounts(S[:,-1])
print kc.shape
#array는 문자열이 섞이면 수도 문자열로 casting.
allFreq=kc[1,:].astype('int').sum()
#vector연산이므로 for-loop가 필요없슴.
prob=kc[1,:].astype('float')/allFreq
print "확률 {0} 전체빈도 {1}".format(prob,allFreq)

#2) 엔트로피를 계산
print "엔트로피는 {0}".format(sum([-p*math.log(p,2) for p in prob]))

#in: array (n x 1 속성컬럼)
#out: 엔트로피값
#교재 calcShannonEnt(dataSet)과 동일한 기능이지만 입력은 1개 컬럼.
def getEntropy1(data):
  kc=findKeyCounts(data)
  allFreq=kc[1,:].astype('int').sum()
  prob=kc[1,:].astype('float')/allFreq
  print "\n\t확률 {0} 전체빈도 {1}".format(prob,allFreq)
  entropy=sum([-p*math.log(p,2) for p in prob])
  print "\t엔트로피는 {0}".format(entropy)
  return entropy

print "함수를 이용한 엔트로피는 {0}".format(getEntropy1(S[:,-1]))

첫째 속성의 키/빈도수 [['middle' 'old' 'young']
 ['5' '5' '5']]
마지막 컬럼 클래스의 키/빈도수 [['No' 'Yes']
 ['6' '9']]
(2, 2)
확률 [ 0.4  0.6] 전체빈도 15
엔트로피는 0.970950594455

	확률 [ 0.4  0.6] 전체빈도 15
	엔트로피는 0.970950594455
함수를 이용한 엔트로피는 0.970950594455


## 3. Information Gain 계산

### 우선 연습

In [6]:
#3) 어느 속성으로 분할할 지 비교
#나이Age의 경우

#H(age)=5/15 * H(age=young) + 5/15 * H(age=middle) + 5/15 * H(age=old)
#=0.888

#컬럼의 조건에 따른 행선택
inData=S
sub=inData[inData[:,0]=='young']
print sub
#array([['young', 'false', 'false', 'fair', 'No'],
#       ['young', 'false', 'false', 'good', 'No'],
#       ['young', 'true', 'false', 'good', 'Yes'],
#       ['young', 'true', 'true', 'fair', 'Yes'],
#       ['young', 'false', 'false', 'fair', 'No']])
print findKeyCounts(sub[:,-1])
#array([['No', 'Yes'],['3', '2']])
freqYoung=len(sub)
entYoung=sum([-p*math.log(p,2) for p in (3/5.,2/5.)])
#0.9709505944546686

sub=inData[inData[:,0]=='middle']
#array([['middle', 'false', 'false', 'fair', 'No'],
#       ['middle', 'false', 'false', 'good', 'No'],
#       ['middle', 'true', 'true', 'good', 'Yes'],
#       ['middle', 'false', 'true', 'excellent', 'Yes'],
#       ['middle', 'false', 'true', 'excellent', 'Yes']]) 
print findKeyCounts(sub[:,-1])
#array([['No', 'Yes'],['2', '3']]) 
freqMiddle=len(sub)
entMiddle=sum([-p*math.log(p,2) for p in (2/5.,3/5.)])
#0.9709505944546686

sub=inData[inData[:,0]=='old']
print findKeyCounts(sub[:,-1])
#array([['No', 'Yes'],['1', '4']])
freqOld=len(sub)
entOld=sum([-p*math.log(p,2) for p in (1/5.,4/5.)])
#0.7219280948873623

probYoung=float(freqYoung)/len(inData)
entPYoung=probYoung*entYoung
print "Young 엔트로피",entPYoung,"=",probYoung,"곱하기",entYoung
entAge=entPYoung

probMiddle=float(freqMiddle)/len(inData)
entPMiddle=probMiddle*entMiddle
print "Middle 엔트로피",entPMiddle,"=",probMiddle,"곱하기",entMiddle
entAge+=entPMiddle

probOld=float(freqOld)/len(inData)
entPOld=probOld*entOld
print "Old 엔트로피",entPOld,"=",probOld,"곱하기",entOld
entAge+=entPOld

print "나이 엔트로피:",entAge,"=",entPYoung,"+",entPMiddle,"+",entPOld
baseEntropy=sum([-p*math.log(p,2) for p in (9/15.,6/15.)])
print "전체 클래스 엔트로피:",baseEntropy
print "나이 information gain:",baseEntropy-entAge,"=",baseEntropy,"-",entAge
#IG(D,age) = 0.971 - 0.888 = 0.083
#IG(D,own_house) = 0.971 - 0.551 = 0.420
#IG(D,has_job) = 0.971 - 0.647 = 0.324
#IG(D,credit_rating) = 0.971 - 0.608 = 0.363
#IG가 제일 큰 own_house가 분기하기 가장 좋은 속성


[['young' 'false' 'false' 'fair' 'No']
 ['young' 'false' 'false' 'good' 'No']
 ['young' 'true' 'false' 'good' 'Yes']
 ['young' 'true' 'true' 'fair' 'Yes']
 ['young' 'false' 'false' 'fair' 'No']]
[['No' 'Yes']
 ['3' '2']]
[['No' 'Yes']
 ['2' '3']]
[['No' 'Yes']
 ['1' '4']]
Young 엔트로피 0.323650198152 = 0.333333333333 곱하기 0.970950594455
Middle 엔트로피 0.323650198152 = 0.333333333333 곱하기 0.970950594455
Old 엔트로피 0.240642698296 = 0.333333333333 곱하기 0.721928094887
나이 엔트로피: 0.887943094599 = 0.323650198152 + 0.323650198152 + 0.240642698296
전체 클래스 엔트로피: 0.970950594455
나이 information gain: 0.0830074998558 = 0.970950594455 - 0.887943094599


### 함수로

In [7]:
inData=S
baseEntropy=getEntropy1(inData[:,-1]) #마지막 열 (클래스)
max_feature=inData.shape[1]-1 #마지막 열(클래스) 제외
InfoGain=np.zeros([max_feature]) #속성별
#속성별 key에 대해 분류 확률(즉, 나이young인 경우 대출yes인 확률)
n_feature=0 # 첫째 속성 나이
keyCounts=findKeyCounts(inData[:,n_feature]) #나이 열의 키값과 빈도
max_key=keyCounts.shape[1] #나이 old,middle,young key는 3개
allFreq=keyCounts[1,:].astype('int').sum() #전체 15개
prob=keyCounts[1,:].astype('float')/allFreq # [5/15,5/15,5/15]
print ">>나이 p={0} allFreq={1}".format(prob,allFreq)
#key(old,middle,young)를 찾아 데이터 잘라서 확률,엔트로피
for n_key in range(max_key):
    keyToSearch=keyCounts[0][n_key]
    subData=inData[inData[:,n_feature]==keyToSearch]
    # 자른 데이터의 클래스 키,빈도 구함
    InfoGain[n_feature]+=prob[n_key]*getEntropy1(subData[:,-1])
    print "keyToSearch={0} 속성엔트로피={1}".format(keyToSearch,InfoGain)
InfoGain[n_feature]=baseEntropy-InfoGain[n_feature]
print "<<나이 InfoGain={0}".format(InfoGain)

#위를 함수로 만들면
def getInfoGain(inData):
    baseEntropy=getEntropy1(inData[:,-1]) #마지막 열 (클래스)
    max_feature=inData.shape[1]-1 #마지막 열(클래스) 제외
    InfoGain=np.zeros([max_feature]) #속성별
    #속성별 key에 대해 분류 확률(즉, 나이young인 경우 대출yes인 확률)
    for n_feature in range(max_feature):
        keyCounts=findKeyCounts(inData[:,n_feature]) #속성 열의 키값과 빈도
        max_key=keyCounts.shape[1] #unique key 값이 몇 개인지
        allFreq=keyCounts[1,:].astype('int').sum()
        prob=keyCounts[1,:].astype('float')/allFreq
        print ">>{0}th col p={1} allFreq={2}".format(n_feature,prob,allFreq)
        #key를 찾아 데이터 잘라서 확률,엔트로피
        for n_key in range(max_key):
            keyToSearch=keyCounts[0][n_key]
            subData=inData[inData[:,n_feature]==keyToSearch]
            # 자른 데이터의 클래스 키,빈도 구함
            InfoGain[n_feature]+=prob[n_key]*getEntropy1(subData[:,-1])
            print "keyToSearch={0} 속성엔트로피={1}".format(keyToSearch,InfoGain)
        InfoGain[n_feature]=baseEntropy-InfoGain[n_feature]
        print "<<{0}th InfoGain={1}".format(n_feature,InfoGain)

getInfoGain(inData)


	확률 [ 0.4  0.6] 전체빈도 15
	엔트로피는 0.970950594455
>>나이 p=[ 0.33333333  0.33333333  0.33333333] allFreq=15

	확률 [ 0.4  0.6] 전체빈도 5
	엔트로피는 0.970950594455
keyToSearch=middle 속성엔트로피=[ 0.3236502  0.         0.         0.       ]

	확률 [ 0.2  0.8] 전체빈도 5
	엔트로피는 0.721928094887
keyToSearch=old 속성엔트로피=[ 0.5642929  0.         0.         0.       ]

	확률 [ 0.6  0.4] 전체빈도 5
	엔트로피는 0.970950594455
keyToSearch=young 속성엔트로피=[ 0.88794309  0.          0.          0.        ]
<<나이 InfoGain=[ 0.0830075  0.         0.         0.       ]

	확률 [ 0.4  0.6] 전체빈도 15
	엔트로피는 0.970950594455
>>0th col p=[ 0.33333333  0.33333333  0.33333333] allFreq=15

	확률 [ 0.4  0.6] 전체빈도 5
	엔트로피는 0.970950594455
keyToSearch=middle 속성엔트로피=[ 0.3236502  0.         0.         0.       ]

	확률 [ 0.2  0.8] 전체빈도 5
	엔트로피는 0.721928094887
keyToSearch=old 속성엔트로피=[ 0.5642929  0.         0.         0.       ]

	확률 [ 0.6  0.4] 전체빈도 5
	엔트로피는 0.970950594455
keyToSearch=young 속성엔트로피=[ 0.88794309  0.          0.          0.        ]
<<0th InfoGain=[ 0.08

## 4. Decision Tree 구조 및 5. 분류

## scikit ML 패키지를 사용하여:

In [11]:
from sklearn import tree
X = [[0, 0], [1, 1]]
Y = [0, 1]
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, Y)
clf.predict([[2., 2.]])


array([1])

## 교재

In [8]:
# learn textbook 엔트로피 계산.
import os
os.chdir('/Users/media/Code/git/else/machinelearninginaction/Ch03')
import trees
myDat,labels=trees.createDataSet()
#myDat [[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]
#labels ['no surfacing', 'flippers']
print "entropy=",trees.calcShannonEnt(myDat) #0.9709505944546686

#inside calcShannonEnt: 학습데이터 1개 행의 클래스의 빈도 계산
numEntries=len(myDat)
labelCounts={}
currentLabel=myDat[0][-1] #[1, 1, 'yes']의 마지막 'yes'
if currentLabel not in labelCounts.keys():
    labelCounts[currentLabel]=0
labelCounts[currentLabel]+=1
print labelCounts #{'yes': 1}

print trees.chooseBestFeatureToSplit(myDat)


# create tree as a dictionary and search
#createTree p.48
myTree=trees.createTree(myDat,labels)
#{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}
print labels #createTree하면 labels lost ['flippers'], so reload!!!

#insdie classify p.56
myDat,labels=trees.createDataSet() #now labels ['no surfacing', 'flippers']
firstStr=myTree.keys()[0] #'no surfacing'
secondDict=myTree[firstStr] #{0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}
featIndex=labels.index(firstStr) #0
secondDict.keys() #[0, 1]
secondDict #{0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}
testVec=[1,0]
key=secondDict.keys()[0] #0
print testVec[featIndex]==key #False
print "testVec={0}을 분류하면 결과 class label={1}".format(testVec,secondDict[0])

# sample data로
S=np.array([['young', 'false', 'false', 'fair', 'No'],
       ['young', 'false', 'false', 'good', 'No'],
       ['young', 'true', 'false', 'good', 'Yes'],
       ['young', 'true', 'true', 'fair', 'Yes'],
       ['young', 'false', 'false', 'fair', 'No'],
       ['middle', 'false', 'false', 'fair', 'No'],
       ['middle', 'false', 'false', 'good', 'No'],
       ['middle', 'true', 'true', 'good', 'Yes'],
       ['middle', 'false', 'true', 'excellent', 'Yes'],
       ['middle', 'false', 'true', 'excellent', 'Yes'],
       ['old', 'false', 'true', 'excellent', 'Yes'],
       ['old', 'false', 'true', 'good', 'Yes'],
       ['old', 'true', 'false', 'good', 'Yes'],
       ['old', 'true', 'false', 'excellent', 'Yes'],
       ['old', 'false', 'false', 'fair', 'No']], 
      dtype='|S9')
dat=S[:,:-1].tolist()
lab=S[:,-1].tolist()
print trees.createTree(dat,lab)

entropy= 0.970950594455
{'yes': 1}
0
['flippers']
False
testVec=[1, 0]을 분류하면 결과 class label=no
{'No': {'middle': {'Yes': {'false': {'No': {'good': 'good', 'fair': 'fair'}}, 'true': {'No': {'true': 'good', 'false': 'excellent'}}}}, 'old': {'No': {'false': {'Yes': {'true': 'good', 'false': 'fair'}}, 'true': {'No': {'good': 'good', 'excellent': 'excellent'}}}}, 'young': {'Yes': {'false': {'No': {'false': 'fair', 'true': 'good'}}, 'true': 'fair'}}}}
